In [1]:
from utils.Ulsan_Event_Data_by_API import * 
from utils.Festival_Data_by_API import * 

### 울산시 행사 데이터

In [18]:
serviceKey = "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D"

In [19]:
params_dict1 = {"serviceKey" : serviceKey,
                "pageNo" : 1,
                "numOfRows" : 1000}

In [4]:
ulsan_event_api = Ulsan_Event_Data_by_API(params_dict = params_dict1)

In [5]:
ulsan_event_data = ulsan_event_api.get()

n_rows : 1000, total_count : 312, max_page = 1


In [6]:
ulsan_event_data.to_csv("/home/seho/Passenger_Demand/data/ulsan_event_data.csv")

### 전국 행사 데이터

In [1]:
import requests
import ast
import xmltodict
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict


class Data_by_API(object):
    
    def __init__(self, url):
        self.url = url
        self.features = None
        self.main_key = None
#         self.serviceKey = serviceKey
    
    
    def calculate_max_page(self, type = "json"):
        rq = self.request()
        
        rq_dict = self.to_dict(txt = rq.text, type = type)
        
        self.n_rows = int(self.params_dict["numOfRows"])
        
        try:
            self.total_count = int(rq_dict["response"]["body"]["totalCount"])
        except:
            xmlsoup = BeautifulSoup(rq.text,'html.parser')
            self.total_count = int(xmlsoup.find("totalcount").text)
                
        max_page = int(np.ceil(self.total_count / self.n_rows))
        
        print(f"n_rows : {self.n_rows}, total_count : {self.total_count}, max_page = {max_page}")
        
        return max_page
    
    
    def create_request_url(self, params_dict):
#         params_dict["service_key"] = self.serviceKey
        params_list = [f"{k}={v}" for k, v in params_dict.items()]
        params_str = "&".join(params_list)
#         print(params_str)
        
        request_url = self.url + params_str
        
        return request_url
    
    
    def create_request_urls(self):
        max_page = self.calculate_max_page(type = self.type)
        
        params_dict = self.params_dict.copy()
        
        request_urls= []
        for i in range(max_page):
            params_dict["pageNo"] = i + 1
            request_urls.append(self.create_request_url(params_dict = params_dict))
            
        return request_urls
    
    
    def to_dict(self, txt, type):
        # json / xml to dict
        if type == "json":
            rq_dict = ast.literal_eval(txt)
        elif type == "xml":
            rq_dict = xmltodict.parse(txt)
            
        return rq_dict
    
    
    def extract_values_from_dict(self, dct):
        try: 
            dict_list = dct["response"]["body"]["items"]["item"]
        except:
            dict_list = dct["response"]["body"]["items"]
            
        return dict_list
    
    
    
    def parse(self, rq, features = None, type = "json"):
        
        data_dict = defaultdict(list)
        
        rq_dict = self.to_dict(txt = rq.text, type = type)

        # 일부 url의 경우는 item이 아닌 items에 값이 존재
        dict_list = self.extract_values_from_dict(dct = rq_dict)
        
        # 값이 1개인 경우 list가 아니라 dictionary 1개가 반환되므로, 이를 list(dict)형태로 변환
        if isinstance(dict_list, dict):
            dict_list = [dict_list]
        
        # item이 없는 경우 빈 Dictionary(data dict)를 반환
        if dict_list is None:
            return data_dict
        
        if features is None:
            features = dict_list[0].keys()
            
        for x in dict_list:
            for col in features:
                data_dict[col].append(x.get(col))

        return data_dict
    
    
    def request(self, request_url = None):
        
        if request_url == None:
            request_url = self.request_url
            
        rq = requests.get(request_url, allow_redirects = True)
        
        return rq
        

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict



class Event_Data_by_API(Data_by_API):
    
    base_url = "http://api.data.go.kr/openapi/tn_pubr_public_pblprfr_event_info_api?" # JSON , XML
    
    def __init__(self, params_dict):
        super().__init__(url = self.base_url)
        self.request_url = super().create_request_url(params_dict = params_dict)
        self.params_dict = params_dict
        self.type = params_dict.get("type").lower()
              
    
    def get(self):
        
        self.request_urls = self.create_request_urls()
        
        data_dict = defaultdict(list)
        for i, request_url in enumerate(self.request_urls):
            rq = self.request(request_url = request_url)
            temp_dict = self.parse(rq = rq, features = None, type = self.type)
            
            for k, v in temp_dict.items():
                data_dict[k].extend(v)
            
        return pd.DataFrame(data_dict)

In [3]:
serviceKey = "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D"

In [4]:
params_dict1 = {"serviceKey" : serviceKey,
                "pageNo" : 1,
                "numOfRows" : 1000,
                "type" : "json"}

In [5]:
event_api = Event_Data_by_API(params_dict = params_dict1)

In [6]:
temp2 = event_api.get() 

n_rows : 1000, total_count : 6396, max_page = 7


### 전국 축제 데이터

In [7]:
params_dict2 = {"serviceKey" : serviceKey,
                "pageNo" : 1,
                "numOfRows" : 1000,
                "type" : "json"}

In [8]:
festival_api = Festival_Data_by_API(params_dict = params_dict2)

In [9]:
festival_data = festival_api.get()

n_rows : 1000, total_count : 1027, max_page = 2


In [16]:
festival_data["fstvlStartDate"] = pd.to_datetime(festival_data["fstvlStartDate"], format = "%Y-%m-%d")
festival_data["fstvlEndDate"] = pd.to_datetime(festival_data["fstvlEndDate"], format = "%Y-%m-%d")
festival_data["referenceDate"] = pd.to_datetime(festival_data["referenceDate"], format = "%Y-%m-%d")

In [32]:
festival_data = festival_data.loc[(festival_data["fstvlStartDate"].dt.year == 2020) & (festival_data["rdnmadr"].str.contains("울산"))]

In [33]:
festival_data.to_csv("/home/seho/Passenger_Demand/data/festival_data.csv")